# Stock Data Reader

## Abstract:
### This module is for financial dataset construction. The functions of this module include:
- Importing daily price data for stocks.
- Importing S&P500 daily index.
- Importing external dataset (daily,anual)
- Importing sentiment data from Quandl (you need to subscribe this service)
- Merging datasets before
- Flexibly constructing features based on the dataset merged
- Customizing the data structure (rolling & un-rolling) and time window for the dataset
### Motivation:
- Constructing dataset could be quite annoying when doing financial researches. You need to collect data from different sources, merge them(This could be more annoying when they have different frequency), and construct the features with different structures and time windows.In this module , you can just adjust several variables, then all these works could be easily achieved.
### Data Source:
- Price and sentiment data are collected from Quandl.com
- S&P500 Index is collected from Yahoo Finance
- There is also a port for importing external data

## Instruction

### For using this module , you must have an API Key for quandl.com. You just need to sign up on the webpage and find API Key in your account setting. If you want to use sentiment data , you have to subscribe "FinSentS Web News Sentiment", the link is https://www.quandl.com/data/NS1-FinSentS-Web-News-Sentiment .

### Import the module and initialize it:

In [1]:
from stock_data_reader import StockDataReader
reader = StockDataReader("your_api_key")

### Read price and sentiment data from quandl:

- In this step, date_start should be one or two years before the start date you finally want to get. Because when you are calculating some features like moving average and Beta,you will need to use data before that final start date.

In [11]:
reader.initialize_data('CF',date_start='2001-01-01',date_end='2016-12-31',sentiment=True)

### Price table:

In [12]:
reader.price_table.head(5)

,date,adj_close,adj_volume
None,,,
0,2005-08-11,2.810457,76671500.0
1,2005-08-12,3.110586,29113500.0
2,2005-08-15,3.001290,10596500.0
3,2005-08-16,2.975268,5141000.0
4,2005-08-17,2.923222,6423500.0


### Sentiment table

In [13]:
reader.sentiment_table.head(5)#sentiment data is only available from 2013

,Date,Sentiment,News Volume,News Buzz
0,2013-01-01,5.0,1.0,10.0
1,2013-01-02,0.0,0.0,0.0
2,2013-01-03,0.0,0.0,0.0
3,2013-01-04,0.0,0.0,0.0
4,2013-01-05,0.0,0.0,0.0


## Read S&P500 Index
- For constructing multiple datasets for different stocks, you just need to import index data once.

In [6]:
reader.read_index('2001-01-01','2016-12-31')

In [7]:
reader.sp500_table.head(5)

,date,sp_adj_close,sp_volume
0,2016-12-30,2238.830078,2.670900e+09
1,2016-12-29,2249.260010,2.336370e+09
2,2016-12-28,2249.919922,2.392360e+09
3,2016-12-27,2268.879883,1.987080e+09
4,2016-12-23,2263.790039,2.020550e+09


## Read external data:
- In this section, the fundamental dataset I use is downloaded from Compustat
- The variable 'y' means that I'm using anual data. You can also set it as 'd' for using daily data. If you want to use hybrid data, please first merge them into daily dataset (this is quite a simple work)  and set the variable as 'd'.
- For constructing multiple datasets for different stocks, you just need to import external data once.

In [8]:
reader.read_external('basic_stats.csv','y')

In [9]:
reader.external_table.head(5) 
# The ticker is not for IBM, but IBM is contained in this dataset.
# This is an anual dataset

,fyear,tic,ceql,csho,ni,sale
0,2001,PNW,2499.323,84.825,312.166,4551.373
1,2002,PNW,2686.153,91.255,149.408,2637.279
2,2003,PNW,2829.779,91.288,240.579,2817.852
3,2004,PNW,2950.196,91.793,243.195,2899.725
4,2005,PNW,3424.964,99.057,176.267,2987.955


## Merge datasets and construct features:

### If you don't want to do any feature construction, you can just merge datasets simply:

In [14]:
dat = reader.merge_table()

In [15]:
dat.iloc[-5:,:]

,date,adj_close,adj_volume,Sentiment,News Volume,News Buzz,tic,ceql,csho,ni,sale,sp_adj_close,sp_volume
2863,2016-12-23,30.445183,5475503.0,0.0,0.0,0.0,CF,3348.0,233.114,-277.0,3685.0,2263.790039,2.020550e+09
2864,2016-12-27,31.674490,6222852.0,0.0,0.0,0.0,CF,3348.0,233.114,-277.0,3685.0,2268.879883,1.987080e+09
2865,2016-12-28,31.357249,3653427.0,-2.0,10.0,3.0,CF,3348.0,233.114,-277.0,3685.0,2249.919922,2.392360e+09
2866,2016-12-29,31.624921,4220127.0,-2.0,3.0,1.0,CF,3348.0,233.114,-277.0,3685.0,2249.260010,2.336370e+09
2867,2016-12-30,31.208543,4181315.0,0.0,0.0,0.0,CF,3348.0,233.114,-277.0,3685.0,2238.830078,2.670900e+09


## If you want to construct features, you don't have to call merge_table(), just simply import a dictionary in format '{'feature_name':function}, and call feature():

In [16]:
def pe(dat):
    eps = dat['ni']/dat['csho']
    return dat['adj_close']/eps
def price_to_book_value(dat):
    book = dat['ceql']/dat['csho']
    return dat['adj_close']/book
def ps(dat):
    sps = dat['sale']/dat['csho']
    return dat['adj_close']/sps
def daily_return(dat):
    return (dat['adj_close']/(dat['adj_close'].shift(1))-1)*100
def average_20(dat):
    moving = dat['adj_close'].rolling(window=21,center=False).mean()
    return moving
def beta(dat):
    ret_price = (dat['adj_close']/(dat['adj_close'].shift(1))-1)*100
    ret_sp = (dat['sp_adj_close']/(dat['sp_adj_close'].shift(1))-1)*100
    beta = (ret_price.rolling(window=252,center=False).cov(ret_sp))/(ret_sp.rolling(window=252,center=False).var())
    return beta
feature_dic = {'pe':pe,
               'price_to_book_value':price_to_book_value,
               'ps':ps,
               '20mean':average_20,
               'return':daily_return,
               'beta':beta}

### It will return an index with all the column names in the dataset

In [18]:
reader.feature(feature_dic)

Index(['date', 'adj_close', 'adj_volume', 'Sentiment', 'News Volume',
       'News Buzz', 'tic', 'ceql', 'csho', 'ni', 'sale', 'sp_adj_close',
       'sp_volume', 'pe', 'price_to_book_value', 'ps', '20mean', 'return',
       'beta'],
      dtype='object')

## Trim the dataset
- You can select out the features which you want to put in the final dataset by calling trim()

In [23]:
feature_list = ['date','adj_close','pe','price_to_book_value','ps','20mean','beta','return']

### It will return an index contains all the features which will appear in your final dataset

In [24]:
reader.trim(feature_list)

Index(['date', 'adj_close', 'pe', 'price_to_book_value', 'ps', '20mean',
       'beta', 'return'],
      dtype='object')


In [25]:
reader.table.iloc[-5:,:]

,date,adj_close,pe,price_to_book_value,ps,20mean,beta,return
2863,2016-12-23,30.445183,-25.621654,2.119832,1.925970,28.752289,1.470747,1.992693
2864,2016-12-27,31.674490,-26.656199,2.205426,2.003736,28.912797,1.471948,4.037773
2865,2016-12-28,31.357249,-26.389219,2.183337,1.983667,29.075194,1.473355,-1.001565
2866,2016-12-29,31.624921,-26.614483,2.201975,2.000600,29.286216,1.462744,0.853620
2867,2016-12-30,31.208543,-26.264073,2.172983,1.974260,29.406125,1.467886,-1.316614


## The last step is to construct the dataset:
- There are two kinds of data structures: rolling data and un-rolling data
- With a time window of 4 days, rolling data contains data 4-days ago, un-rolling data contains data from yesterday to 4-days ago.

In [26]:
# trim the dataset to make it more clear
reader.trim(['date','adj_close','pe'])

Index(['date', 'adj_close', 'pe'], dtype='object')


In [28]:
dat1 = reader.construct_data(rolling=False, time_window=5,thresh_date='2009-01-01')

In [30]:
dat1.head(5)

,date,adj_close,5day_adj_close,5day_pe
854,2009-01-02,9.415645,8.095383,0.572235
855,2009-01-05,9.937079,8.216524,0.580798
856,2009-01-06,9.703575,8.198968,0.579557
857,2009-01-07,9.354197,8.362245,0.591098
858,2009-01-08,9.598235,8.630862,0.610086


In [31]:
dat2 = reader.construct_data(rolling=True,time_window = 3,thresh_date='2012-01-01')
dat2.head(8)

,date,adj_close,1day_adj_close,1day_pe,2day_adj_close,2day_pe,3day_adj_close,3day_pe
1610,2012-01-03,27.443501,25.868011,1.099475,25.691371,1.091967,25.380912,1.078771
1611,2012-01-04,28.189317,27.443501,0.934492,25.868011,1.099475,25.691371,1.091967
1612,2012-01-05,28.487286,28.189317,0.959888,27.443501,0.934492,25.868011,1.099475
1613,2012-01-06,28.189317,28.487286,0.970035,28.189317,0.959888,27.443501,0.934492
1614,2012-01-09,28.512265,28.189317,0.959888,28.487286,0.970035,28.189317,0.959888
1615,2012-01-10,29.655968,28.512265,0.970885,28.189317,0.959888,28.487286,0.970035
1616,2012-01-11,29.825471,29.655968,1.009830,28.512265,0.970885,28.189317,0.959888
1617,2012-01-12,29.670242,29.825471,1.015602,29.655968,1.009830,28.512265,0.970885
